In [ ]:
!pip install numpy
!pip install tensorflow
!pip install keras
!pip install nltk
!pip install regex
!pip install contractions
!pip install scikit-learn
!pip install pandas
!pip install keras-preprocessing


import numpy as np
import pandas as pd 
import re
from keras.preprocessing.text import Tokenizer 
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, GRU 
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import contractions
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from keras_preprocessing.sequence import pad_sequences

In [3]:
#!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz
#!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_All_Electronics.json.gz
import gzip
import json

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

data = getDF('reviews_Electronics_5.json.gz')

In [4]:
data

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AI9M1HBD188VK,0324322402,Brandy,"[1, 1]",This device worked very nicely for the large c...,4.0,Clicker,1327881600,"01 30, 2012"
1,A3E7PG9CHDBICA,059403390X,Amazonian,"[0, 0]",A beautiful cover that feels great to hold in ...,5.0,Color Cover,1403827200,"06 27, 2014"
2,A32JCI4AK2JTTG,059403390X,"Darryl Bennett ""Fuzzy342""","[0, 2]",I think that this is a very nice and economica...,5.0,Nook Simple Touch Industiral Cover.,1356480000,"12 26, 2012"
3,A15J2ESU6EMNTE,059403390X,J. VanderZiel,"[1, 2]",This is a good quality case and was reasonably...,4.0,Good cover but does *not* act as support for Nook,1357171200,"01 3, 2013"
4,A2BTN4KV00ZKTF,059403390X,Richard W.,"[0, 0]",I bought this for my wife to protect her Nook ...,5.0,Very sleek and stylish cover,1400198400,"05 16, 2014"
...,...,...,...,...,...,...,...,...,...
72816,A2EJNU4G5G0NEY,B00KPX7PBO,J.A.,"[2, 3]",Well this was a surprise! Looking for int'l c...,5.0,Extremely pleased with this service!!,1404950400,"07 10, 2014"
72817,AEWZEQW0ZLTS0,B00KPX7PBO,Tim Bushra,"[2, 3]",I just received the service instruction card i...,5.0,Crystal Clear Voice/ Cheap Rates,1401840000,"06 4, 2014"
72818,A3V6TYXFMPDOAY,B00L1MTZ20,June,"[1, 2]",I just purchased two of these new designed cab...,1.0,DOES NOT WORK WITH NOOK COLOR,1404086400,"06 30, 2014"
72819,AOGI3TISX5IK4,B00L5DWGF8,"Bill Atkinson ""TechHead""","[0, 0]",Does not split a stereo signal to two mono (L ...,1.0,One Star,1404691200,"07 7, 2014"


In [5]:
#remove html tags from text
def remove_html_tags_text(string):
    result = re.sub('<.*?>','',string)
    return result

data['reviewText']=data['reviewText'].apply(lambda x : remove_html_tags_text(x))
data

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AI9M1HBD188VK,0324322402,Brandy,"[1, 1]",This device worked very nicely for the large c...,4.0,Clicker,1327881600,"01 30, 2012"
1,A3E7PG9CHDBICA,059403390X,Amazonian,"[0, 0]",A beautiful cover that feels great to hold in ...,5.0,Color Cover,1403827200,"06 27, 2014"
2,A32JCI4AK2JTTG,059403390X,"Darryl Bennett ""Fuzzy342""","[0, 2]",I think that this is a very nice and economica...,5.0,Nook Simple Touch Industiral Cover.,1356480000,"12 26, 2012"
3,A15J2ESU6EMNTE,059403390X,J. VanderZiel,"[1, 2]",This is a good quality case and was reasonably...,4.0,Good cover but does *not* act as support for Nook,1357171200,"01 3, 2013"
4,A2BTN4KV00ZKTF,059403390X,Richard W.,"[0, 0]",I bought this for my wife to protect her Nook ...,5.0,Very sleek and stylish cover,1400198400,"05 16, 2014"
...,...,...,...,...,...,...,...,...,...
72816,A2EJNU4G5G0NEY,B00KPX7PBO,J.A.,"[2, 3]",Well this was a surprise! Looking for int'l c...,5.0,Extremely pleased with this service!!,1404950400,"07 10, 2014"
72817,AEWZEQW0ZLTS0,B00KPX7PBO,Tim Bushra,"[2, 3]",I just received the service instruction card i...,5.0,Crystal Clear Voice/ Cheap Rates,1401840000,"06 4, 2014"
72818,A3V6TYXFMPDOAY,B00L1MTZ20,June,"[1, 2]",I just purchased two of these new designed cab...,1.0,DOES NOT WORK WITH NOOK COLOR,1404086400,"06 30, 2014"
72819,AOGI3TISX5IK4,B00L5DWGF8,"Bill Atkinson ""TechHead""","[0, 0]",Does not split a stereo signal to two mono (L ...,1.0,One Star,1404691200,"07 7, 2014"


In [6]:
#removing shortened words like "I'll" etc.
def text_cleaner(string):
    result  = contractions.fix(string)
    return  result.lower()

data['reviewText']=data['reviewText'].apply(lambda x : text_cleaner(x))


In [ ]:
#removing stopwords
stop_words = set(stopwords.words('english'))
data['reviewText'] = data['reviewText'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
data

In [8]:
max_text_len=50
max_summary_len=8

In [9]:
cleaned_text =np.array(data['reviewText'])
cleaned_summary=np.array(data['summary'])

In [10]:

short_text=[]
short_summary=[]

for i in range(len(cleaned_text)):
    if(len(cleaned_summary[i].split())<=max_summary_len and len(cleaned_text[i].split())<=max_text_len):
        short_text.append(cleaned_text[i])
        short_summary.append(cleaned_summary[i])
        
df=pd.DataFrame({'text':short_text,'summary':short_summary})

In [11]:
df['summary'] = df['summary'].apply(lambda x : x.lower() )

In [12]:
df['summary'] = df['summary'].apply(lambda x : 'start '+ x + ' end')

In [ ]:
df['summary']=df['summary'].apply(lambda x : text_cleaner(x))
df['summary'] = df['summary'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
df

In [14]:
x_tr,x_val,y_tr,y_val=train_test_split(np.array(df['text']),np.array(df['summary']),random_state=0,shuffle=True) 

In [15]:
x_tokenizer = Tokenizer() 
x_tokenizer.fit_on_texts(list(x_tr))

In [16]:
cnt_of_unique = 4

cnt = 0
tot_cnt = 0

for key, value in x_tokenizer.word_counts.items():
    tot_cnt = tot_cnt + 1
    if value < cnt_of_unique:
        cnt = cnt + 1



In [17]:
x_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
x_tokenizer.fit_on_texts(list(x_tr))
x_tr_seq    =   x_tokenizer.texts_to_sequences(x_tr) 
x_val_seq   =   x_tokenizer.texts_to_sequences(x_val)
x_tr    =   pad_sequences(x_tr_seq,  maxlen=max_text_len, padding='post')
x_val   =   pad_sequences(x_val_seq, maxlen=max_text_len, padding='post')
x_voc   =  x_tokenizer.num_words + 1

In [19]:
y_tokenizer = Tokenizer()
y_tokenizer.fit_on_texts(list(y_tr))

In [20]:
y_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
y_tokenizer.fit_on_texts(list(y_tr))
y_tr_seq    =   y_tokenizer.texts_to_sequences(y_tr) 
y_val_seq   =   y_tokenizer.texts_to_sequences(y_val) 
y_tr    =   pad_sequences(y_tr_seq, maxlen=max_summary_len, padding='post')
y_val   =   pad_sequences(y_val_seq, maxlen=max_summary_len, padding='post')
y_voc  =   y_tokenizer.num_words +1

In [21]:
ind=[]
for i in range(len(y_tr)):
    cnt=0
    for j in y_tr[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_tr=np.delete(y_tr,ind, axis=0)
x_tr=np.delete(x_tr,ind, axis=0)

In [22]:
ind=[]
for i in range(len(y_val)):
    cnt=0
    for j in y_val[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_val=np.delete(y_val,ind, axis=0)
x_val=np.delete(x_val,ind, axis=0)

In [ ]:
#second model with 2x Encoder - decoder LSTM
from keras import backend as K 
import attention

latent_dim = 10
embedding_dim = 5
rec_dropOut = 0.2
dropOut = 0.2
batchSize = 1024
epstoTrain = 50
encoder_inputs = Input(shape=(max_text_len,))
enc_emb = Embedding(x_voc, embedding_dim, trainable=True)(encoder_inputs)
encoder_lstm1 = LSTM(latent_dim, return_sequences=True, 
                     return_state=True, 
                     dropout=dropOut, 
                     recurrent_dropout = rec_dropOut)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)
encoder_lstm2 = LSTM(latent_dim, return_sequences=True, 
                     return_state=True, 
                     dropout=dropOut, 
                     recurrent_dropout = rec_dropOut)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(y_voc, embedding_dim, trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm1 = LSTM(latent_dim, return_sequences=True,
                     return_state=True, dropout=dropOut,
                     recurrent_dropout =rec_dropOut)
decoder_output1, _, _ = decoder_lstm1(dec_emb, initial_state=[state_h1, state_c1])
decoder_lstm2 = LSTM(latent_dim, return_sequences=True,
                     return_state=True, dropout=dropOut,
                     recurrent_dropout =rec_dropOut)
decoder_output2, _, _ = decoder_lstm2(decoder_output1, initial_state=[state_h2, state_c2])
attn_layer = attention.AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_output2, decoder_output2])
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_output2, attn_out])
decoder_dense = TimeDistributed(Dense(y_voc, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
model.summary()


In [ ]:
#first model with 1x Encoder - decoder LSTM
from keras import backend as K 
import attention


#tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

#tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

#with tpu_strategy.scope():
    
    #used hyperparameters
    #100/50, 300/200 500,350, 1000/500
    #02/02, 02/02, 02/02 02/02

latent_dim = 100
embedding_dim = 50
rec_dropOut = 0.2
dropOut = 0.2
batchSize = 512
epstoTrain = 50
encoder_inputs = Input(shape=(max_text_len,))
enc_emb = Embedding(x_voc, embedding_dim, trainable=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_sequences=True, 
                    return_state=True, 
                    dropout=dropOut, 
                    recurrent_dropout = rec_dropOut)
encoder_output, state_h, state_c = encoder_lstm(enc_emb)
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(y_voc, embedding_dim, trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True,
                    return_state=True, dropout=dropOut,
                    recurrent_dropout =rec_dropOut)
decoder_output, _, _ = decoder_lstm(dec_emb, initial_state=[state_h, state_c])
attn_layer = attention.AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_output, decoder_output])
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_output, attn_out])
decoder_dense = TimeDistributed(Dense(y_voc, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
model.summary()

In [30]:
es = [EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)]



In [ ]:
history=model.fit([x_tr, x_tr[:,:-1]], x_tr[:,1:], epochs=epstoTrain, callbacks=[es], batch_size=batchSize, validation_data=([x_val, x_val[:,:-1]], x_val[:,1:]))



In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [36]:
model_json = model.to_json()
with open("4thModel.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("4thModel.h5")

Saved model to disk


In [37]:
#model.load_weights("4thModel.h5")

In [ ]:
from matplotlib import pyplot
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [38]:
reverse_target_word_index=y_tokenizer.index_word
reverse_source_word_index=x_tokenizer.index_word
target_word_index=y_tokenizer.word_index

In [48]:
#decoder for 2nd, 3th and 4th model

encoder_model = Model(inputs=encoder_inputs, outputs=[encoder_output2, state_h2, state_c2])
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_text_len, latent_dim))
dec_emb2 = dec_emb_layer(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm1(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])
decoder_outputs2 = decoder_dense(decoder_inf_concat)
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input, decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [ ]:
#decoder for 1st model

encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_output, state_h, state_c])

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_text_len,latent_dim))
dec_emb2= dec_emb_layer(decoder_inputs) 
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])
decoder_outputs2 = decoder_dense(decoder_inf_concat) 
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [49]:
def decode_sequence(input_seq):
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = target_word_index['start']
    decoded_sentence = ''
    while True:
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        if sampled_token_index in reverse_target_word_index:
            sampled_token = reverse_target_word_index[sampled_token_index]
        else:
            sampled_token = "<>"
        if sampled_token == 'end' or len(decoded_sentence.split()) >= (max_summary_len-1):
            break
        decoded_sentence += ' ' + sampled_token
        target_seq[0, 0] = sampled_token_index
        e_h, e_c = h, c

    return decoded_sentence.strip()


In [50]:

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString

In [ ]:
for i in range(0,10):
    print("Review:",seq2text(x_tr[i]))
    print("Predicted summary:",decode_sequence(x_tr[i].reshape(1,max_text_len)))
    print("\n")

In [52]:
def remove_stopwords(text):
    stopwords_set = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stopwords_set]
    return ' '.join(filtered_words)


In [ ]:
#custom text summarization 
input = "Insert your custom text here"
input = text_cleaner(input)
input = remove_html_tags_text(input)
input = remove_stopwords(input)
inputseq = pad_sequences(x_tokenizer.texts_to_sequences([input]), max_text_len)
generated_summary = decode_sequence(inputseq)

generated_summary




In [ ]:
!pip install rouge
from rouge import Rouge

model_summary = []
reference_summary = []
for i in range(0,100):
    reference_summary.append(seq2text(x_tr[i]))
    model_summary.append(decode_sequence(x_tr[i].reshape(1,max_text_len)))


# Remove empty strings from model_summary and reference_summary
model_summary = [s for s in model_summary if s]
reference_summary = [s for s in reference_summary if s]

rouge = Rouge()
total_scores = {'rouge-1': {'f': 0, 'p': 0, 'r': 0}, 'rouge-2': {'f': 0, 'p': 0, 'r': 0}, 'rouge-l': {'f': 0, 'p': 0, 'r': 0}}
for i in range(len(model_summary)):
    score = rouge.get_scores(model_summary[i], reference_summary[i])
    total_scores['rouge-1']['f'] += score[0]['rouge-1']['f']
    total_scores['rouge-1']['p'] += score[0]['rouge-1']['p']
    total_scores['rouge-1']['r'] += score[0]['rouge-1']['r']
    total_scores['rouge-2']['f'] += score[0]['rouge-2']['f']
    total_scores['rouge-2']['p'] += score[0]['rouge-2']['p']
    total_scores['rouge-2']['r'] += score[0]['rouge-2']['r']
    total_scores['rouge-l']['f'] += score[0]['rouge-l']['f']
    total_scores['rouge-l']['p'] += score[0]['rouge-l']['p']
    total_scores['rouge-l']['r'] += score[0]['rouge-l']['r']

# Compute the average ROUGE scores
num_pairs = len(model_summary)
avg_scores = {'rouge-1': {'f': total_scores['rouge-1']['f']/num_pairs, 'p': total_scores['rouge-1']['p']/num_pairs, 'r': total_scores['rouge-1']['r']/num_pairs},
              'rouge-2': {'f': total_scores['rouge-2']['f']/num_pairs, 'p': total_scores['rouge-2']['p']/num_pairs, 'r': total_scores['rouge-2']['r']/num_pairs},
              'rouge-l': {'f': total_scores['rouge-l']['f']/num_pairs, 'p': total_scores['rouge-l']['p']/num_pairs, 'r': total_scores['rouge-l']['r']/num_pairs}}









In [ ]:
# Print the average ROUGE scores
print(avg_scores)